In [1]:
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
from biogeme.expressions import (
    Beta,
    Variable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
)

In [2]:
np.random.seed(seed=90267)
df = pd.read_csv('F:/more work/netherlands/wave8data/model/nested_logit/nested_data_4mode.csv')


In [3]:
df.drop(['AFSTV', 'G_AFSTAND', 'KAFSTV', 'KHVM', 'KVERTTIJD', 'REISDUUR', 'KREISDUUR', 'REISDUUR_ORG', 'G_REISDUUR', 'ROLAUTO', 'PARKEERKOSTEN', 'KRVM', 'Bestuurder', 'HHBRUTOINK2_w5','CarStatus' , 'TransitStatus'] , axis=1 , inplace=True)
df.dropna(inplace=True)
df=df[df["PROV_27"]==1]
df["ID"] = df["ID"]*0.0001
df["CarTravelTime"] = df["CarTravelTime"]*0.0001
df["BikeDistance"] = df["BikeDistance"]*0.00001
df["TransitDistance"] = df["TransitDistance"]*0.00001
df["TransitDuration"] = df["TransitDuration"]*0.0001
df["alokatie_station"] = df["alokatie_station"]*0.0001
df["alokatie_bushalte2xpu"] = df["alokatie_bushalte2xpu"]*0.0001
df["vlokatie_centrumsgw"] = df["vlokatie_centrumsgw"]*0.0001

database = db.Database('first_logit_model', df)
globals().update(database.variables)
database.panel("ID")


In [4]:
# Parameters to be estimated
#car as a driver
ASC_car = Beta('ASC_car', 0, None, None, 0)
N_ACTIVITEIT13_1_car = Beta('N_ACTIVITEIT13_1_car', 0, None, None, 0)
#WERKSITUATIE_ANDERS1_car = Beta('WERKSITUATIE_ANDERS1_car', 0, None, None, 0)
vlokatie_centrumsgw_car = Beta('vlokatie_centrumsgw_car', 0, None, None, 0)
KAFSTV_3_car = Beta('KAFSTV_3_car', 0, None, None, 0)
#GEBRUIK_PARK2_car = Beta('GEBRUIK_PARK2_car', 0, None, None, 0)
KLEEFT2_9_car = Beta('KLEEFT2_9_car', 0, None, None, 0)
INVLOED3_car = Beta('INVLOED3_car', 0, None, None, 0)
DOEL_8_car = Beta('DOEL_8_car', 0, None, None, 0)
N_KIND_2_car = Beta('N_KIND_2_car', 0, None, None, 0)
##EVENT4_car = Beta('EVENT4_car', 0, None, None, 0)





# train
ASC_bus_Train = Beta('ASC_bus_Train', 0, None, None, 1)
#TransitDuration_bus_Train = Beta('TransitDuration_bus_Train', 0, None, None, 0)
CarTravelTime_bus_Train = Beta('CarTravelTime_bus_Train', 0, None, None, 0)
#KAFSTV_12_bus_Train = Beta('KAFSTV_12_bus_Train', 0, None, None, 0)
PAUTO_bus_Train = Beta('PAUTO_bus_Train', 0, None, None, 0)
MOTIEF_11_bus_Train = Beta('MOTIEF_11_bus_Train', 0, None, None, 0)
KAFSTV_12_bus_Train = Beta('KAFSTV_12_bus_Train', 0, None, None, 0)

#N_KIND






# bicycle
ASC_Scooter_Bicycle = Beta('ASC_Bicycle', 0, None, None, 0)
MOTIEF_5_Bicycle = Beta('MOTIEF_5_Bicycle', 0, None, None, 0)
TYPE_OVKAART4_Bicycle = Beta('TYPE_OVKAART4_Bicycle', 0, None, None, 0)
KLEEFT2_8_Bicycle = Beta('KLEEFT2_8_Bicycle', 0, None, None, 0)
KAFSTV_2_Bicycle = Beta('KAFSTV_2_Bicycle', 0, None, None, 0)
#EVENT9_Bicycle = Beta('EVENT9_Bicycle', 0, None, None, 0)
TYPE_OVKAART2_Bicycle = Beta('TYPE_OVKAART2_Bicycle', 0, None, None, 0)
EVENT6_Bicycle = Beta('EVENT6_Bicycle', 0, None, None, 0)
new_Bicycle = Beta('new_Bicycle', 0, None, None, 0)









# walking
ASC_Walking = Beta('ASC_Walking', 0, None, None, 0)
TransitDuration_Walking = Beta('TransitDuration_Walking', 0, None, None, 0)
HERKOMST_w5_2_Walking = Beta('HERKOMST_w5_2_Walking', 0, None, None, 0)
new_Walking = Beta('new_Walking', 0, None, None, 0)

new_mixed = Beta('new_mixed', 0, None, None, 0)
new_mixed2 = Beta('new_mixed2', 0, None, None, 0)


In [5]:

# Define a random parameter, normally distributed, designed to be used
# for Monte-Carlo simulation
#EVENT5_m = Beta('EVENT5_m', 1, None, None, 0)

# It is advised not to use 0 as starting value for the following parameter.
vlokatie_centrumsgw_S = Beta('vlokatie_centrumsgw_S', 1, None, None, 0)
#vlokatie_centrumsgw_S_RND = vlokatie_centrumsgw_car + N_ACTIVITEIT13_1_car*N_ACTIVITEIT13_1 + vlokatie_centrumsgw_S * bioDraws('vlokatie_centrumsgw_S_RND', 'NORMAL')WERKSITUATIE_ANDERS1+ROLAUTO_2
vlokatie_centrumsgw_S_RND = vlokatie_centrumsgw_car + new_mixed*MOTIEF_7 + new_mixed2*ROLAUTO_2 +  vlokatie_centrumsgw_S * bioDraws('vlokatie_centrumsgw_S_RND', 'NORMAL')

In [6]:
# Definition of the utility functions
V1 = ASC_car + vlokatie_centrumsgw_S_RND*vlokatie_centrumsgw + KAFSTV_3_car*KAFSTV_3  

V2 = ASC_bus_Train + PAUTO_bus_Train*PAUTO + KAFSTV_12_bus_Train*KAFSTV_12

V3 = ASC_Scooter_Bicycle + MOTIEF_5_Bicycle*MOTIEF_5 + KLEEFT2_8_Bicycle*KLEEFT2_8 + KAFSTV_2_Bicycle*KAFSTV_2

V4 = ASC_Walking + HERKOMST_w5_2_Walking*HERKOMST_w5_2 + TransitDuration_Walking*TransitDuration

In [7]:
# Associate utility functions with the numbering of alternatives
V = {1: V1, 2: V2, 3: V3, 4: V4}
# Associate the availability conditions with the alternatives
av = {1: av_car_as_driver, 2: av_train, 3: av_Bicycle , 4: av_Walking}

In [8]:
obsprob = models.logit(V, av, Choice_modals)

# Conditional to the random parameters, the likelihood of all observations for
# one individual (the trajectory) is the product of the likelihood of
# each observation.
condprobIndiv = PanelLikelihoodTrajectory(obsprob)

# We integrate over the random parameters using Monte-Carlo
logprob = log(MonteCarlo(condprobIndiv))


In [9]:
# Define level of verbosity
#logger = msg.bioMessage()
# logger.setSilent()
# logger.setWarning()
# logger.setGeneral()
#logger.setDetailed()
# logger.setDebug()

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob, numberOfDraws=1000)
biogeme.modelName = 'mixed_analysislogit12'

# Estimate the parameters.
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)


The use of argument numberOfDraws in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_draws in section MonteCarloof the .toml parameter file. The default file name is biogeme.toml


                            Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_Bicycle             -1.702491      0.313864    -5.424301  5.818166e-08
ASC_Walking              0.839529      0.092858     9.041003  0.000000e+00
ASC_car                 -3.696469      0.628416    -5.882202  4.048435e-09
HERKOMST_w5_2_Walking   -1.209174      0.236861    -5.104988  3.308163e-07
KAFSTV_12_bus_Train      0.679815      0.153498     4.428826  9.474737e-06
KAFSTV_2_Bicycle         1.941576      0.664410     2.922255  3.475072e-03
KAFSTV_3_car             1.497411      0.574412     2.606859  9.137700e-03
KLEEFT2_8_Bicycle        1.061744      0.477561     2.223265  2.619796e-02
MOTIEF_5_Bicycle         2.404951      0.929092     2.588497  9.639580e-03
PAUTO_bus_Train         -0.476233      0.170689    -2.790068  5.269697e-03
TransitDuration_Walking -1.370243      0.293274    -4.672233  2.979423e-06
new_mixed                1.843862      0.542715     3.397480  6.800963e-04
new_mixed2               

In [10]:
print(results.shortSummary())
print(results.getEstimatedParameters(onlyRobust=False))

Results for model mixed_analysislogit12
Nbr of parameters:		15
Sample size:			69
Observations:			563
Excluded data:			0
Final log likelihood:		-546.4229
Akaike Information Criterion:	1122.846
Bayesian Information Criterion:	1156.357

                            Value   Std err    t-test       p-value  \
ASC_Bicycle             -1.702491  0.196883 -8.647217  0.000000e+00   
ASC_Walking              0.839529  0.183801  4.567589  4.933667e-06   
ASC_car                 -3.696469  0.503454 -7.342224  2.100542e-13   
HERKOMST_w5_2_Walking   -1.209174  0.498631 -2.424987  1.530895e-02   
KAFSTV_12_bus_Train      0.679815  0.301766  2.252791  2.427235e-02   
KAFSTV_2_Bicycle         1.941576  0.582245  3.334637  8.541089e-04   
KAFSTV_3_car             1.497411  0.719031  2.082539  3.729325e-02   
KLEEFT2_8_Bicycle        1.061744  0.552948  1.920151  5.483887e-02   
MOTIEF_5_Bicycle         2.404951  0.745566  3.225670  1.256782e-03   
PAUTO_bus_Train         -0.476233  0.208643 -2.282528  2